In [5]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, Dense, Reshape, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
def make_model():
    model = Sequential()
    vgg16 = VGG16(include_top=False, input_shape=(32, 32, 3))
    vgg16.trainable = False
    model.add(vgg16)
#     model.add(Conv2D(filters=100, kernel_size=5, activation="relu", input_shape=(32, 32, 3) ))
#     model.add(MaxPool2D(pool_size=2))
#     model.add(BatchNormalization())
    
#     model.add(Conv2D(filters=128, kernel_size=3, activation="relu"))
#     model.add(MaxPool2D(pool_size=2))
#     model.add(BatchNormalization())
    ##green = concatenate 
    model.add(Flatten())
    
    model.add(Dense(512, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(512, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(10, activation="softmax"))
    return model
model = make_model()
model.compile(loss="sparse_categorical_crossentropy", optimizer=Nadam(learning_rate=1e-4 ), metrics=["accuracy"])

early_stopping = EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True)
reduceLR = ReduceLROnPlateau(monitor="val_accuracy", factor=0.3, patience=6)
def change_lr(epoch_idx, lr):
    if epoch_idx==10:
        return lr*0.5
    if epoch_idx==10:
        return lr*0.3
    return lr
lr_scheduler = LearningRateScheduler(change_lr)

generator = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.15, horizontal_flip=True, validation_split=0.15)

model.fit(generator.flow(x_train, y_train, batch_size=128),
          validation_data=generator.flow(x_train, y_train, batch_size=128, subset="validation"),
          steps_per_epoch=len(x_train)//128, epochs=30, verbose=1, validation_split=0.15, callbacks=[early_stopping, reduceLR])
model.evaluate(x_test, y_test)

Epoch 1/30
  4/333 [..............................] - ETA: 6:53 - loss: 3.5823 - accuracy: 0.0781

KeyboardInterrupt: 